In [1]:
from src.MOF_builder.vlx_mof_builder_v2 import MofBuilder

mof = MofBuilder()
mof.mof_family='UiO-67'
mof.node_metal='Zr'

mof.linker_xyz_file='ndi.xyz'

mof.set_rotation_optimizer_display(True)

mof.set_use_saved_optimized_rotations_npy('rota')
mof.set_use_saved_rotations_as_initial_guess(False)


#mof.use_saved_optimized_rotations_npy('rota')
#save optimized rotations to numpy file for later use
mof.save_optimized_rotations('rota')
mof.set_supercell_cleaved_buffer_minus(0.2)
mof.supercell=(1,1,1)
mof.build()
mof.write_gromacs_files()
mof.show(res_indices=True,res_names=False)

Optimized rotations are loaded from:  rota.npy
UiO-67 selected
available metal nodes: ['Zr', 'Hf']
please select a metal node
will search template cif files in /Users/chenxili/GitHub/vlx_mofbuilder/data/template_database
fcu.cif is found in template_database
fcu.cif will be used for MOF building
Zr node is selected
found the file including ['12c', 'Zr']
File copied from /Users/chenxili/GitHub/vlx_mofbuilder/data/nodes_database/12c_Zr.pdb to nodes/12c_Zr.pdb
nodes/12c_Zr.pdb is fetched
default node without dummy atoms will be usednodes/12c_Zr.pdb
nodes will be saved in nodes
found the file including ['12c', 'Zr']
2 is selected for linker splitting
center is two points
[9, 14]
ditopic linker: center are two points
find connected X in edge:   24
find connected X in edge:   39
pairXs: [24, 39]
linker_center_frag: 46 [18, 33]
linker edge fragment edges/diedge.pdb is saved and will be used for MOF building
--------------------------------------------------------------------------------
     

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [2]:
#mof.remove_linkers = [33]
#mof.remove_nodes = [1]
mof.make_defects_missing(clean_unsaturated_linkers=True,update_node_term=True)
mof.write_defective_model_gromacs_file()
mof.show_defective_model(res_indices=True,res_names=False)

saved_eG is saved nodes:  28 edges:  33
main fragment of the MOF cell is kept
defective eG is updated nodes:  362 edges:  774
defect_gro_name is not set, will be saved as:  defective_mof_UiO-67_ndi.gro
writing defective gro file
defective_mof_UiO-67_ndi.gro is saved
node_res_num:  4
edge_res_num:  6
term_res_num:  36
showing the gromacs file output_gros/defective_mof_UiO-67_ndi


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [1]:
from src.MOF_builder._learn_template import extract_type_atoms_fcoords_in_primitive_cell,extract_unit_cell,make_supercell_3x3x3,clust_analysis_points,filter_overlapping_points
import numpy as np
template_cif_file = "mil101_cleaved.cif"
#eenode, _, _ = extract_cluster_center_from_templatecif(
#    template_cif_file, "Y", 6,cluster_distance_threshhold=4
#) 

cell_info, array_atom, array_target_atoms = (
        extract_type_atoms_fcoords_in_primitive_cell(template_cif_file, 'Cr')
    )
unit_cell = extract_unit_cell(cell_info)
    # unit_cell = np.round(unit_cell,3)
metal333 = make_supercell_3x3x3(array_target_atoms)
metal333 = np.vstack(metal333)
    # cluster analysis in cartesian coordinates
array_metal_ccords = np.dot(unit_cell, metal333.T).T
#for each atom, find the closest 2 atoms, these 3 atoms are the cluster, then find the center of the cluster, one atom can only be in one cluster

array = array_metal_ccords
centers = []
matched_atoms = []
for i in range(len(array)):
    if i in matched_atoms:
        continue
    atom = array[i]
    dist = np.linalg.norm(array-atom,axis=1)
    idx = np.argsort(dist)
    cluster = array[idx[:3]]
    matched_atoms.extend(idx[:3])
    center = np.mean(cluster,axis=0)
    centers.append(center)
    print(len(matched_atoms)/3)
#remove overlapping points
cluster_centers_ccoords = np.vstack(centers)
# cluster_centers should return fractional coordinates
cluster_centers_fcoords = np.dot(
    np.linalg.inv(unit_cell), cluster_centers_ccoords.T
).T
# filter cluster centers which is inside the unit cell, boundary condition is [-0.01,1.01]
# cluster_centers_fcoords = np.round(cluster_centers_fcoords,3)
cluster_centers_fcoords = np.mod(cluster_centers_fcoords, 1)
# cluster_centers_fcoords = [c for c in cluster_centers_fcoords if all([i>=-0.01 and i<=1.01 for i in c])]

# cluster_centers_fcoords = np.unique(cluster_centers_fcoords,axis=0)


apply 192  symmetry operation
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
172.0
173.0
174.0
175.0
176.0
177.0
178.0
179.0
180.

In [194]:
#use np.isclose to filter overlapping points
cluster_centers_fcoords_neat = cluster_centers_fcoords[0:280]
for i in range(280,cluster_centers_fcoords.shape[0]):
    if not any(np.isclose(cluster_centers_fcoords[i],cluster_centers_fcoords_neat,atol=0.07).all(axis=1)):
        cluster_centers_fcoords_neat = np.vstack([cluster_centers_fcoords_neat,cluster_centers_fcoords[i]])


print(cluster_centers_fcoords_neat.shape)
print(cluster_centers_fcoords.shape)

#cluster_centers_fcoords_neat = np.round(cluster_centers_fcoords_neat, 4)
np.save("vvnode.npy",cluster_centers_fcoords_neat)

(309, 3)
(7551, 3)


In [195]:
unit_cell =np.array([[ 100.0, 0.0, 0.0],
 [ 0.0, 100.0, 0.0],
 [ 0.0, 0.0, 100.0]])

In [196]:
#make 333 supercell of vvnode, for each line, looking for closest 6 atoms, distance range in cartesian coordinates shoud be around 12-15
#then find 6 center of the node in the 6 pairs
#remove the overlapping points by remove the lines by index


vvv333node = make_supercell_3x3x3(cluster_centers_fcoords_neat)
vvv333node = np.vstack(vvv333node)
    # cluster analysis in cartesian coordinates
array_vvvnode_ccords = np.dot(unit_cell, vvv333node.T).T
matched_pairs = []
centers = []
overlap=[]
check =[]
#for each atom, find the closest 6 atoms, matched pair only once
for i in range(len(cluster_centers_fcoords_neat)):
    idxs = None
    dist = None
    atom = array_vvvnode_ccords[i]
    dist = np.linalg.norm(array_vvvnode_ccords-atom,axis=1)
    idxs = np.argsort(dist)
    #check distance range of 
    if dist[idxs[7]] > 15 and dist[idxs[6]] <15 and dist[idxs[1]]>11.1:
        #find each pair center
        indices = idxs[1:7]
        for id in indices:
            if (i,id) in matched_pairs:
                continue
            matched_pairs.append((id,i))
            matched_pairs.append((i,id))
            center = np.mean([array_vvvnode_ccords[i],array_vvvnode_ccords[id]],axis=0)
            centers.append(center)
    else:
        
        #remove overlapping points
        if dist[idxs[1]]<10:
            if idxs[1] > i and idxs[1] < len(cluster_centers_fcoords_neat):
                overlap.append(idxs[1])
                print("error",i,dist[idxs[7]],dist[idxs[1]],dist[idxs[6]],idxs[:8])
        if dist[idxs[2]]<10:
            if idxs[2] > i and idxs[2] < len(cluster_centers_fcoords_neat):
                overlap.append(idxs[2])
                print("error",i,dist[idxs[7]],dist[idxs[1]],dist[idxs[6]],idxs[:8])
        if dist[idxs[3]]<10:
            if idxs[3] > i and idxs[3] < len(cluster_centers_fcoords_neat):
                overlap.append(idxs[3])
                print("error",i,dist[idxs[7]],dist[idxs[1]],dist[idxs[6]],idxs[:8])
        #if dist[idxs[7]]<15.5:
        #    if idxs[7] > i and idxs[7] < len(cluster_centers_fcoords_neat):
        #        check.append((i,idxs[7],dist[idxs[7]]))
        #        print("error",i,dist[idxs[7]],dist[idxs[1]],dist[idxs[6]],idxs[:8])
#
        #if dist[idxs[6]]>15.5:
        #    if idxs[6] > i and idxs[6] < len(cluster_centers_fcoords_neat):
        #        check.append((i))
        #        print("error",i,dist[idxs[7]],dist[idxs[1]],dist[idxs[6]],idxs[:8])





            





error 14 13.343038134797713 9.273977931107364 13.293607486307115 [  14  297 1428   59  137 1376   80  112]
error 19 13.367705695277547 2.1316282072803006e-14 13.125082010325798 [  19  152 3083  266 3031 2911  200 2856]
error 23 13.125082010325801 1.4210854715202004e-14 13.12508201032579 [  23  276 3084 3077   59 4206  188 3065]
error 25 13.367705695277534 2.0097183471152322e-14 13.125082010325805 [  25  155 3087  249 3046  135 2985   69]
error 29 13.367705695277534 9.133211678021901 13.125082010325784 [ 29 292 893 264 866 175 279 126]
error 68 13.293607486307115 9.273977931107387 13.293607486307106 [  68  305 5783  109 5716   37  130 5602]
error 73 13.367705695277547 1.4210854715202004e-14 13.12508201032579 [ 73 210 595 573 232 335 172 434]
error 79 13.36770569527754 1.4210854715202004e-14 13.1250820103258 [ 79 213 599 263 539  33 485 119]
error 91 13.343038134797725 9.273977931107355 13.293607486307115 [  91  299 1503  169  320  570 2029   34]
error 118 13.343038134797723 9.2739779311

In [197]:
cluster_centers_fcoords_neat = cluster_centers_fcoords_neat[[r for r in range(len(cluster_centers_fcoords_neat)) if r not in overlap]]

In [ ]:
len(cluster_centers_fcoords_neat),np.save("vvnode.npy",np.vstack(cluster_centers_fcoords_neat))

(272, None)

In [200]:
#final check
vvv333node = make_supercell_3x3x3(cluster_centers_fcoords_neat)
vvv333node = np.vstack(vvv333node)
    # cluster analysis in cartesian coordinates
array_vvvnode_ccords = np.dot(unit_cell, vvv333node.T).T
matched_pairs = []
centers = []
overlap=[]
check =[]
#for each atom, find the closest 6 atoms, matched pair only once
for i in range(len(cluster_centers_fcoords_neat)):
    idxs = None
    dist = None
    atom = array_vvvnode_ccords[i]
    dist = np.linalg.norm(array_vvvnode_ccords-atom,axis=1)
    idxs = np.argsort(dist)
    #check distance range of 
    if dist[idxs[7]] > 15 and dist[idxs[6]] <15 and dist[idxs[1]]>11.1:
              #find each pair center
        indices = idxs[1:7]
        for id in indices:
            if (i,id) in matched_pairs:
                continue
            matched_pairs.append((id,i))
            matched_pairs.append((i,id))
            center = np.mean([array_vvvnode_ccords[i],array_vvvnode_ccords[id]],axis=0)
            centers.append(center)
    else:
        print("error",i,dist[idxs[7]],dist[idxs[1]],dist[idxs[6]],idxs[:8])

In [66]:
np.linalg.norm(array_vvvnode_ccords[244:246]-array_vvvnode_ccords[0],axis=1)

array([13.34303813, 96.99508006])

In [202]:
#use mod to unique the centers
centers = np.vstack(centers)
centers_fccords = np.dot(np.linalg.inv(unit_cell),centers.T).T
centers_fccords = np.mod(centers_fccords, 1)
centers_fccords = np.unique(centers_fccords,axis=0)
centers_fccords = np.round(centers_fccords,4)
np.save("eenode.npy",centers_fccords)

In [ ]:
from src.MOF_builder._learn_template import extract_type_atoms_fcoords_in_primitive_cell,extract_unit_cell,make_supercell_3x3x3,clust_analysis_points,filter_overlapping_points
import numpy as np
template_cif_file = "mil101_cleaved.cif"
#eenode, _, _ = extract_cluster_center_from_templatecif(
#    template_cif_file, "Y", 6,cluster_distance_threshhold=4
#) 

cell_info, array_atom, array_target_atoms = (
        extract_type_atoms_fcoords_in_primitive_cell(template_cif_file, 'Y')
    )
unit_cell = extract_unit_cell(cell_info)
    # unit_cell = np.round(unit_cell,3)
metal333 = make_supercell_3x3x3(array_target_atoms)
metal333 = np.vstack(metal333)
    # cluster analysis in cartesian coordinates
array_metal_ccords = np.dot(unit_cell, metal333.T).T


apply 192  symmetry operation


In [4]:
#for each atom, find the closest 5 atoms, these 6 atoms are the cluster, then find the center of the cluster, one atom can only be in one cluster

array = array_metal_ccords
centers = []
matched_atoms = []
for i in range(len(array)):
    if i in matched_atoms:
        continue
    atom = array[i]
    dist = np.linalg.norm(array-atom,axis=1)
    idx = np.argsort(dist)
    cluster = array[idx[:6]]
    matched_atoms.extend(idx[:6])
    center = np.mean(cluster,axis=0)
    centers.append(center)
    print(len(matched_atoms)/6)
#remove overlapping points

    



1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
172.0
173.0
174.0
175.0
176.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
184.0
185.

In [5]:
cluster_centers_ccoords = np.vstack(centers)
# cluster_centers should return fractional coordinates
cluster_centers_fcoords = np.dot(
    np.linalg.inv(unit_cell), cluster_centers_ccoords.T
).T
# filter cluster centers which is inside the unit cell, boundary condition is [-0.01,1.01]
# cluster_centers_fcoords = np.round(cluster_centers_fcoords,3)
cluster_centers_fcoords = np.mod(cluster_centers_fcoords, 1)
# cluster_centers_fcoords = [c for c in cluster_centers_fcoords if all([i>=-0.01 and i<=1.01 for i in c])]
cluster_centers_fcoords = filter_overlapping_points(cluster_centers_fcoords, 0.001)
cluster_centers_fcoords = np.round(cluster_centers_fcoords, 3)
# cluster_centers_fcoords = np.unique(cluster_centers_fcoords,axis=0)

In [8]:

np.save("eenode.npy",cluster_centers_fcoords)

##### done: defects and postprocess of node residues split 
##### done: make defects on eG and add terminations 
##### done: add linker exchange defects by superimpose the x ;linker_termination can be realized by linker exchange;
##### done: split dummy nodes by use dummy dict info ;

##### fixed: dummy atom recognizer bug
##### postponed: write dict even with dummy node False to split node
##### done: multitopic test
##### postponed: bridge node and virtual edge test 

In [1]:
from src.MOF_builder.vlx_mof_builder_v2 import MofBuilder 
mof = MofBuilder()
mof.mof_family = 'PCN-222'
mof.node_metal = 'Zr'
mof.linker_xyz_file ='noCoTCPP.xyz'
mof.save_optimized_rotations('rotc')

#mof.set_use_saved_optimized_rotations_npy('rotc')
#mof.set_use_saved_rotations_as_initial_guess(True)

mof.supercell = (1,1,1)
mof.build()
mof.write_gromacs_files()
mof.show(res_indices=True,res_names=False)

PCN-222 selected
available metal nodes: ['Zr', 'Hf']
please select a metal node
will search template cif files in /Users/chenxili/GitHub/vlx_mofbuilder/data/template_database
csq.cif is found in template_database
csq.cif will be used for MOF building
Zr node is selected
found the file including ['8c', 'Zr']
File copied from /Users/chenxili/GitHub/vlx_mofbuilder/data/nodes_database/8c_Zr_1_Ch.pdb to nodes/8c_Zr_1_Ch.pdb
nodes/8c_Zr_1_Ch.pdb is fetched
default node without dummy atoms will be usednodes/8c_Zr_1_Ch.pdb
nodes will be saved in nodes
found the file including ['8c', 'Zr']
4 is selected for linker splitting
center is a cycle
[0, 1, 20, 6, 5, 7, 23, 11, 10, 12, 22, 17, 15, 16, 21, 2]
tritopic/tetratopic/multitopic: center is a cycle
find connected X in edge frag 33 30 20
find connected X in edge frag 39 36 23
find connected X in edge frag 45 42 22
find connected X in edge frag 27 24 21
center_frag: 32 [20, 21, 22, 23]
outer_frag: 10 [0, 3]
linker center fragment nodes/tetracente

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           81     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  8.00347D+05    |proj g|=  2.04658D+01

At iterate    1    f=  8.00346D+05    |proj g|=  4.92437D+00

At iterate    2    f=  8.00346D+05    |proj g|=  2.33995D+00

At iterate    3    f=  8.00346D+05    |proj g|=  6.86850D-01

At iterate    4    f=  8.00346D+05    |proj g|=  5.82077D-01

At iterate    5    f=  8.00346D+05    |proj g|=  5.47152D-01
--------------------------------------------------------------------------------

                     rotations optimization completed                     
--------------------------------------------------------------------------------
At iterate    6    f=  8.00346D+05    |proj g|=  9.31323D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searche

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [2]:
mof.remove_linkers=[10,11]
mof.remove_nodes=[1]
mof.make_defects_missing(clean_unsaturated_linkers=False,update_node_term=True)
mof.write_defective_model_gromacs_file()
mof.show_defective_model(res_indices=True,res_names=False)

saved_eG is saved nodes:  16 edges:  24
main fragment of the MOF cell is kept
defective eG is updated nodes:  94 edges:  187
defect_gro_name is not set, will be saved as:  defective_mof_PCN-222_noCoTCPP.gro
writing defective gro file
defective_mof_PCN-222_noCoTCPP.gro is saved
node_res_num:  9
edge_res_num:  5
term_res_num:  53
showing the gromacs file output_gros/defective_mof_PCN-222_noCoTCPP


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [1]:
from src.MOF_builder.vlx_mof_builder_v2 import MofBuilder 
mof = MofBuilder()
mof.mof_family='MIL-101'
mof.node_metal='Cr'
#mof.preparation.use_dummy_node(True)

mof.linker_xyz_file='bdc.xyz'
mof.supercell=(1,1,1)
mof.set_connection_constant_length(1.1)

mof.set_use_saved_optimized_rotations_npy('rotmil102')
mof.set_use_saved_rotations_as_initial_guess(False)

#save optimized rotations to numpy file for later use
mof.save_optimized_rotations('rotmil102')

#mof.set_supercell_cleaved_buffer_minus(-0.02)
#mof.set_supercell_cleaved_buffer_plus(0.02)
mof.build()
mof.write_gromacs_files()
mof.show(res_names=False,res_indices=False)

Optimized rotations are loaded from:  rotmil102.npy
MIL-101 selected
available metal nodes: ['Cr']
please select a metal node
will search template cif files in /Users/chenxili/GitHub/vlx_mofbuilder/data/template_database
mil101.cif is found in template_database
mil101.cif will be used for MOF building
Cr node is selected
found the file including ['6c', 'Cr']
File copied from /Users/chenxili/GitHub/vlx_mofbuilder/data/nodes_database/6c_Cr.pdb to nodes/6c_Cr.pdb
nodes/6c_Cr.pdb is fetched
default node without dummy atoms will be usednodes/6c_Cr.pdb
nodes will be saved in nodes
found the file including ['6c', 'Cr']
2 is selected for linker splitting
center is a cycle
[0, 1, 2, 3, 4, 5]
ditopic linker: center is a cycle
Xs in the center cycle
pairXs: [0, 3]
linker_center_frag: 10 [0, 3]
linker edge fragment edges/diedge.pdb is saved and will be used for MOF building
--------------------------------------------------------------------------------
                     Preparation is complete

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [2]:
mof.make_defects_missing(clean_unsaturated_linkers=True,update_node_term=True)
mof.write_defective_model_gromacs_file()
mof.show_defective_model(res_names=False,res_indices=False)

saved_eG is saved nodes:  1109 edges:  2310
main fragment of the MOF cell is kept
defective eG is updated nodes:  9871 edges:  21069
defect_gro_name is not set, will be saved as:  defective_mof_MIL-101_bdc.gro
writing defective gro file
defective_mof_MIL-101_bdc.gro is saved
node_res_num:  296
edge_res_num:  750
term_res_num:  276
showing the gromacs file output_gros/defective_mof_MIL-101_bdc


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [8]:
import numpy as np
axis = np.array([5,3,2])
axis = axis / np.linalg.norm(axis)
print(axis)
flip=np.eye(3) - 2 * np.outer(axis, axis) 
print(np.dot(flip,axis))

[0.81110711 0.48666426 0.32444284]
[-0.81110711 -0.48666426 -0.32444284]


In [3]:
mof.net.unsaturated_linker

[]